![image.png](https://i.imgur.com/a3uAqnb.png)

## 💡 **Introduction**

In this lab, we explore how to build, train, and interpret a **neural network for regression** using **PyTorch**.  
Our goal is to predict the **aqueous solubility (`logS`)** of chemical compounds based on their molecular descriptors.  
Through this hands-on exercise, we demonstrate the complete workflow of a machine learning project — from data preprocessing to model evaluation and interpretation.

The dataset, obtained from the **Delaney solubility dataset**, contains key molecular properties such as:
- `MolLogP`: Hydrophobicity measure (log partition coefficient)  
- `MolWt`: Molecular weight  
- `NumRotatableBonds`: Structural flexibility indicator  
- `AromaticProportion`: Ratio of aromatic atoms  

By combining these features, we train a neural network to learn the non-linear relationship between molecular structure and solubility.

### 🧠 **Importing Libraries and Frameworks**

In this step, we import all the essential Python libraries that will be used throughout the lab:

- **NumPy** (`np`) — for numerical operations such as array manipulation.  
- **Pandas** (`pd`) — for data handling and preprocessing using DataFrames.  
- **Matplotlib** (`plt`) — for visualizing data and results.  
- **PyTorch** (`torch`, `torch.nn`, `torch.optim`) — for building, training, and optimizing neural networks.  
- **Dataset** and **DataLoader** — from PyTorch’s `torch.utils.data` to manage our data efficiently during training.  
- **Scikit-learn** (`sklearn`) — specifically:
  - `train_test_split` for splitting data into training and testing sets.
  - `StandardScaler` for normalizing features to improve model convergence.

These imports set up the foundational tools we’ll need for creating and training our neural network model.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### 📊 **Loading and Preparing the Dataset**

In this section, we load and preprocess the dataset used for training our neural network.

1. **Dataset Source:**  
   The data is loaded directly from a public GitHub repository. It contains molecular descriptors and solubility values (`logS`) for various compounds.  

2. **Feature and Target Selection:**  
   - **Features (`X`)**:  
     - `MolLogP`: Logarithm of the partition coefficient (hydrophobicity).  
     - `MolWt`: Molecular weight.  
     - `NumRotatableBonds`: Number of rotatable bonds in the molecule.  
     - `AromaticProportion`: Ratio of aromatic atoms to total heavy atoms.  
   - **Target (`y`)**:  
     - `logS`: The solubility of the molecule (log scale).

3. **Data Splitting:**  
   We split the dataset into **training (80%)** and **validation (20%)** sets using `train_test_split` to evaluate our model's generalization.

4. **Normalization:**  
   Since features vary in scale, we use `StandardScaler` to normalize them — ensuring each feature has **zero mean** and **unit variance**, which helps the neural network converge more efficiently.

Finally, the shapes of the training and validation sets are printed to confirm successful preprocessing.


In [ ]:
# --- Load dataset ---
data = pd.read_csv(
    'https://raw.githubusercontent.com/dataprofessor/data/master/delaney_solubility_with_descriptors.csv'
)

feature_cols = ["MolLogP", "MolWt", "NumRotatableBonds", "AromaticProportion"]
target_col   = "logS"

X = data[feature_cols].values
y = data[[target_col]].values

# --- Split (train/val) ---
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- Normalize using sklearn StandardScaler ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)

print("Train shape:", X_train_scaled.shape, "Val shape:", X_val_scaled.shape)


### 🧩 **Creating a Custom PyTorch Dataset**

Here we define a custom dataset class named `SolubilityDataset`, which extends PyTorch’s built-in `Dataset` class.  
This allows us to efficiently handle tabular data (`X`, `y`) and later feed it into a `DataLoader` for batching and shuffling.

#### 🔍 **Class Breakdown**
- **`__init__(self, X, y)`**  
  - Converts input NumPy arrays (`X`, `y`) into PyTorch tensors of type `float32`.  
  - This ensures compatibility with PyTorch models and GPU computation.

- **`__len__(self)`**  
  - Returns the total number of samples in the dataset.  
  - Used internally by PyTorch to know how many batches can be created.

- **`__getitem__(self, idx)`**  
  - Retrieves one sample (features and target) at a given index `idx`.  
  - Enables easy iteration over data during training and validation.

This structure standardizes how data is accessed — making it clean, reusable, and compatible with PyTorch’s training pipeline.


In [ ]:
class SolubilityDataset(Dataset):
    """Basic dataset for tabular (X, y) regression."""
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

### 🧺 **Creating DataLoaders for Training and Validation**

Now that we’ve defined our custom dataset, we use PyTorch’s `DataLoader` to handle batching and shuffling of our data.

#### ⚙️ **Steps Explained**

1. **Batch Size:**  
   We define `batch_size = 32`, meaning the model will process 32 samples at a time before updating its weights.

2. **Dataset Creation:**  
   - `train_dataset` and `val_dataset` are created using our `SolubilityDataset` class.  
   - They wrap the preprocessed training and validation data respectively.

3. **DataLoader Setup:**  
   - `train_loader`: Loads training data in batches of 32 and **shuffles** it each epoch to improve learning.  
   - `val_loader`: Loads validation data in batches of 32 **without shuffling**, since evaluation doesn’t require randomization.

4. **Sanity Check:**  
  prints the shape of one batch of features (`X`) and targets (`y`) to confirm the DataLoader works correctly.

This setup enables efficient data feeding to the neural network during both training and evaluation.


In [ ]:
batch_size = 32

train_dataset = SolubilityDataset(X_train_scaled, y_train)
val_dataset   = SolubilityDataset(X_val_scaled,   y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)

# quick sanity check
xb, yb = next(iter(train_loader))
print("Batch X:", xb.shape, "Batch y:", yb.shape)



### 🧱 **Defining the Neural Network (Layer-by-Layer)**

In this section, we build the `SolubilityNet` model **manually layer by layer.

#### 🧩 **Architecture Overview**

| Layer | Type | Input → Output | Purpose |
|:------|:------|:----------------|:--------|
| 1 | `Linear(4, 32)` | 4 → 32 | First dense layer taking molecular features as input |
| 2 | `LeakyReLU()` | — | Non-linear activation preventing dead neurons |
| 3 | `Dropout(0.2)` | — | Regularization layer to reduce overfitting |
| 4 | `Linear(32, 16)` | 32 → 16 | Second dense layer |
| 5 | `LeakyReLU()` | — | Adds non-linearity again |
| 6 | `Dropout(0.2)` | — | Further regularization |
| 7 | `Linear(16, 8)` | 16 → 8 | Third dense layer |
| 8 | `LeakyReLU()` | — | Activation to introduce non-linear patterns |
| 9 | `Linear(8, 1)` | 8 → 1 | Output layer producing the solubility prediction (`logS`) |

#### ⚙️ **Forward Pass**
The `forward()` method defines how input data flows through each layer —  
from raw molecular descriptors to the final predicted solubility value.

This explicit implementation helps you understand **how data transforms step by step** inside the network.


In [ ]:
class SolubilityNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Define layers explicitly
        self.fc1 = nn.Linear(4, 16)
        self.act1 = nn.LeakyReLU()
        self.drop1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(16, 8)
        self.act2 = nn.LeakyReLU()

        self.fc3 = nn.Linear(8, 1)

    def forward(self, x):
        x = self.drop1(self.act1(self.fc1(x)))  # Input → Hidden layer 1
        x = self.act2(self.fc2(x))               # Hidden layer 2
        x = self.fc3(x)                          # Output layer
        return x

model = SolubilityNet()
print(model)


### ⚙️ **Loss Function and Optimizer**

We set up how the model learns and updates its weights during training.

- **Loss Function:**  
  `nn.MSELoss()` measures the average squared difference between predicted and actual solubility (`logS`) values — suitable for regression.

- **Optimizer:**  
  `optim.AdamW(model.parameters(), lr=0.01)` updates weights using the AdamW algorithm, which combines adaptive learning rates with weight decay to improve stability and prevent overfitting.

These choices help the model minimize prediction error efficiently during training.


In [ ]:
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.005)


### 🔁 **Training the Neural Network**

In this section, we train our model to learn the relationship between molecular descriptors and solubility (`logS`).

#### 🧠 **Training Setup**
- **Epochs:** We train for `25` complete passes over the training data.  
- **Trackers:** Two lists (`train_losses`, `val_losses`) store average losses for each epoch to monitor progress.

#### ⚙️ **Training Loop**
For each epoch:
1. **Training Phase (`model.train()`):**
   - Loops through all batches from `train_loader`.
   - Performs forward pass → computes loss → backpropagates gradients → updates weights.
   - Collects average training loss for the epoch.

2. **Validation Phase (`model.eval()`):**
   - Disables gradient tracking using `torch.no_grad()`.
   - Evaluates the model on unseen validation data.
   - Computes and stores the average validation loss.

Finally, each epoch prints both training and validation losses, allowing us to track how well the model is learning and whether it’s overfitting or improving steadily.


In [ ]:
epochs = 25
train_losses, val_losses = [], []

for epoch in range(epochs):
    # ---- Training phase ----
    model.train()
    batch_train_losses = []

    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        batch_train_losses.append(loss.item())

    train_losses.append(np.mean(batch_train_losses))

    # ---- Validation phase ----
    model.eval()
    batch_val_losses = []
    with torch.no_grad():
        for xb, yb in val_loader:
            preds = model(xb)
            vloss = criterion(preds, yb)
            batch_val_losses.append(vloss.item())
    val_losses.append(np.mean(batch_val_losses))
    print(f"Epoch {epoch:03d} | Train {train_losses[-1]:.4f} | Val {val_losses[-1]:.4f}")


### 📈 **Visualizing Training and Validation Loss**

After training, we plot the loss curves to observe how the model’s performance evolved over time.

- **Training Loss (Train MSE):** Indicates how well the model fits the training data.  
- **Validation Loss (Val MSE):** Shows how well the model generalizes to unseen data.

#### 🔍 **Interpretation**
- A smooth **decrease in both losses** suggests effective learning.  
- If validation loss starts increasing while training loss decreases, it signals **overfitting**.

This plot helps confirm whether the model converged properly and provides insights into its learning dynamics.


In [ ]:
plt.figure()
plt.plot(train_losses, label="Train MSE")
plt.plot(val_losses, label="Val MSE")
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.title("Training Curve — Solubility Prediction")
plt.legend()
plt.show()


### 🧩 **Feature Importance via Permutation Test**

To understand which molecular descriptors most influence solubility predictions, we use a **permutation importance** approach.

#### ⚙️ **How It Works**
1. Compute the **base loss** on the validation set (normal, unshuffled data).  
2. For each feature in `feature_cols`:
   - Shuffle that feature’s values while keeping others fixed.  
   - Recalculate the model’s loss (`perm_loss`).  
   - The **increase in loss (ΔLoss)** indicates how important that feature is —  
     a larger ΔLoss means the feature had a stronger impact on predictions.

#### 🧠 **Interpretation**
By comparing how much the loss rises when each feature is shuffled, we can rank features by their predictive power.  
This provides a simple yet effective way to interpret the neural network’s behavior on tabular data.


In [ ]:
base_loss = criterion(model(torch.tensor(X_val_scaled, dtype=torch.float32)),
                      torch.tensor(y_val, dtype=torch.float32)).item()

for i, name in enumerate(feature_cols):
    X_perm = X_val_scaled.copy()
    np.random.shuffle(X_perm[:, i])
    with torch.no_grad():
        perm_loss = criterion(model(torch.tensor(X_perm, dtype=torch.float32)),
                              torch.tensor(y_val, dtype=torch.float32)).item()
    print(f"{name:20s} ΔLoss: {perm_loss - base_loss:.4f}")


### 🧾 **Evaluating Model Predictions**

In this step, we evaluate how well the trained model predicts solubility (`logS`) on the **validation set**.

#### ⚙️ **Process Overview**
1. Switch the model to **evaluation mode** using `model.eval()` — this disables dropout and other training-only behaviors.  
2. Use `torch.no_grad()` to perform inference efficiently without tracking gradients.  
3. Generate predictions (`y_pred_val`) on the validation features.  
4. Print a small comparison table showing the **true vs predicted** solubility values for the first few samples.

This provides a quick and intuitive check of the model’s performance on unseen data before moving to more detailed metrics or visualizations.


In [ ]:
# --- Evaluate model on validation set ---
model.eval()
with torch.no_grad():
    X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
    y_pred_val = model(X_val_tensor)

# --- Print results ---
print(f"{'True logS':>12} | {'Predicted logS':>14}")
print("-" * 29)

for i in range(min(5, len(y_val_tensor))):
    print(f"{y_val_tensor[i, 0]:12.3f} | {y_pred_val[i, 0]:14.3f}")



## 🧾 **Summary**

In this experiment, we:
1. **Loaded and preprocessed** the dataset using `pandas` and `scikit-learn`, applying normalization for better training stability.  
2. **Created a custom PyTorch dataset and DataLoader** to efficiently handle batching and shuffling of data.  
3. **Built a feedforward neural network** (`SolubilityNet`) with multiple dense layers, LeakyReLU activations, and dropout regularization.  
4. **Trained the model** using the **Mean Squared Error (MSE)** loss function and the **AdamW optimizer** over multiple epochs, tracking both training and validation losses.  
5. **Visualized training progress** using a loss curve to confirm proper convergence and detect potential overfitting.  
6. **Analyzed feature importance** using permutation testing to understand which molecular descriptors most affected solubility predictions.

Overall, this lab provided a clear, end-to-end example of **regression with neural networks in PyTorch**, emphasizing both **model implementation** and **interpretability**, a crucial aspect in real-world scientific applications.